# Creazione di un Physics-Engine in C/C++

Nella computer science una branca molto importante di essa è sicuramente quella legate alla simulazione dei fenomeni fisici. In generale, la simulazione di questi avviene tramite dei metodi numerici a teorie già esistenti per ottenere il risultato più accurato possibile a quello che si vede nel mondo reale. 
Questo consente, ad esempio, ai programmatori di videogiochi di predire e analizzare correttamente come qualcosa si comporterà prima ancora di averlo di fatto "costruito" (si intende, con questa costruzione del periodo abbastanza oscura, di fissare il comportamento delle cose che includerà nel suo gioco prima ancora di aver effettivamente costruito).

## Leggi di Newton

Nei videogiochi vogliamo animare degli oggetti che, difficilmente, si comportano come dei *punti materiali* (anche se a noi fisici piacerebbe molto) ma come dei corpi rigidi. Prima però di arrivare a ciò, fissiamo dei concetti molto utili che ci serviranno più avanti.

Come si produce l'animazione di un moto? Si produce una sequenza di immagine in una successione e, se queste sono mostrare "velocemente", queste daranno l'idea di un movimento continuo e regolare. Dunque dobbiamo modificare molto frequentemente lo stato fisico del nostro sistema, in accordo con le leggi teoriche utilizzate e dal tipo di fenomeno che si sta cercando di rappresentare.

Da qua dunque si arriva a definire il concetto di **physics engine**: una componente del software di un computer che performa la simulazione. Riceve delle specifiche riguardo ai corpi che si sta cercando di simulare (con dei parametri di configurazione) e la simulazione può essere eseguita per "passi", di una frazione di secondo.
**Nota bene:** l'engine si occupa solamente di effettuare la simulazione numerica. Cosa dovrà essere fatto con i calcoli non è compito dell'engine

Il moto dei corpi può essere modellato utilizzando la meccanica newtoniana (anche se in fondo ci occuperemo anche della fisica con vincoli, che permette anche la formulazione di una meccanica lagrangiana) che può essere formulata tramite le tre leggi di Newton:
1) Inerzia: un corpo permane nel suo stato di quiete o di moto rettilineo uniforme se la risultante delle forze applicate all'oggetto è nulla;
2) $\vec{F}=m\vec{a}$: la risultante delle forze applicate ad un oggetto è proporzionale alla massa;
3) Legge di azione e reazione: se un corpo A esercita una forza su un corpo B pari a $\vec{F}_{A \to B}$ allora B esercita una forza opposta e uguale in modulo al corpo A

Tramite queste leggi, è possibile rappresentare qualunque moto presente in natura


![metodo_simulativo](https://assets.toptal.io/images?url=https%3A%2F%2Fuploads.toptal.io%2Fblog%2Fimage%2F765%2Ftoptal-blog-image-1421917038776.png&width=1920)


Ora, per capire come funziona una simulazione è necessario ovviamente utilizzare i vettori o, più precisamente, un sistema di assi dove la posizione di un corpo verrà indicata da una terna di coordinate
$$
    \vec{r}(t_0) = x_0 \hat{x} + y_0 \hat{y} + z_0 \hat{z} 
$$

### Punto materiale

Per entrare più in profondità con l'argomento, possiamo iniziare a gettare le fondamento del nostro PhysicEngine simulando delle particelle, che risultano essere molto più semplici da trattare rispetto ad una simulazione del corpo rigido. Una particella può essere definita come un punto nello spazio a cui è assegnata una massa, un vettore posizione e una velocità e, in accordo con la prima legge di Newton, la sua velocità cambierà esclusivamente se una forza è applicato su di esso. Quando la velocità ha norma diversa da zero, la posizione cambierà nel tempo.

Come simuliamo delle particelle? Possiamo pensare di considerare un array di particelle in uno stato iniziale e con una massa fissata (almeno per adesso). Successivamente, andiamo a computare tutte le forze agenti sulla particella $i$-esima e si va a calcolare l'accelerazione risultante dall'applicazione di queste, andando a calcolare posizione e velocità. Se lavoriamo con "step" di tempo molto piccoli possiamo affermare che

$$
\begin{cases}
    &\vec{r}(t_{i+1}) = \vec{r}(t_i + dt) = \vec{r_{t_i}} + \vec{v}(t_i)dt \\
    &\vec{a}(t_{i+1}) = \vec{a}(t_i+dt) = \vec{a_{t_i}} + \frac{\vec{F_{t_i}}}{m_i} \\
    &\vec{v}(t_{i+1}) = \vec{v}(t_i + dt) = \vec{v_{t_i}} + \vec{a_{t_i}}dt \\
\end{cases}
$$

che alla fine non è tanto diverso da quello che si fa quando si cerca di risolvere un'equazione differenziale con librerie come $\text{\texttt{simpy}}$ in Python con librerie come $\text{\texttt{odeint()}}$.

### Corpo rigido

Un corpo rigido è un oggetto in cui la distanza fra i suoi punti non varia. Pertanto possiamo tranquillamente affermare che si tratta di un oggetto non deformabile e, dunque, intuire che si tratta dell'astrazione più naturale (che possiamo fare) al momento per rappresentare gli oggetti che ci circondano (chiaramente è un'astrazione, come possiamo banalmente osservare, per il semplice fatto che pur prendendo l'oggetto più indeformabile di questo mondo, se è soggetto alla più misera forza, questo comunque si deformerà, anche se di poco).

Naturalmente il corpo rigido è, in un certo senso, un'estensione di un punto materiale, siccome entrambi condividono il fatto di avere una massa, una posizione e una velocità; tuttavia possiede anche un volume e una forma, dunque può ruotare (e ciò quindi aumenta la complessità in quello che dobbiamo andare a rappresentare). Per indicare la posizione di un corpo rigido, solitamente, si "riporta" quella del centro di massa, che può essere calcolata nella seguente maniera se immaginiamo il corpo rigido come costituito da tante piccole particelle di massa "piccola":
$$
    x_{CM} = \frac{\sum\limits_i m_i r_i}{M}
$$
dove $m_i$ rappresenta la massa della $i$-esima particella, $r_i$ rappresenta il raggio vettore che punta dall'origine del sistema di riferimento all'$i$-esima particella mentre $M = \sum\limits_i m_i$ ovvero è la massa del corpo rigido. In realtà, i corpi rigidi sono "continui" e non "discreti", dunque per calcolare il centro di massa dovremo utilizzare l'integrale (immaginando il nostro corpo come costituito sempre da delle particelle piccole con massa $dm$, potremmo pensare che $dm = p(\vec{r})dV$):
$$
    x_{CM} = \frac{\int p(\vec{r})\vec{r}dV}{M}
$$
dove invece $M=\int dm = \int p(\vec{r})dV$ dove la funzione $p(\vec{r})$ restituisce la densità in funzione della distanza.
Siccome il corpo rigido ruota, bisogna introdurre delle grandezze angolari che siano l'equivalente di quelle lineari. In due dimensioni, un corpo rigido può ruotare attorno ad un asse anche se per ora ci concentreremo del moto senza vincoli, dunque l'unico asse attorno a cui può ruotare è quello del CM.
Per guadagnare una velocità angolare, il nostro corpo riceve una forza "angolare" che indichiamo con l'espressione *momento di una forza*:
$$
\tau = \vec{r} \wedge \vec{F}
$$
tuttavia, per la seconda cardinale, sappiamo che
$$
    \sum\limits_i \tau_i = \frac{d\vec{L}}{dt} \stackrel{\text{in 2 dimens.}}{=} I \vec{\alpha}
$$
dove $\alpha$ è l'accelerazione angolare. (Nota bene: per questa relazione abbiamo ragionato come se ci trovassimo nel centro di massa, dunque il termine aggiuntivo che dovrebbe comparite $\vec{v_0} \wedge \vec{P}$ dove $\vec{v_0}$ è la velocità del centro di massa e $\vec{P}$ è la quantità di moto del sistema, ma questo termine è zero siccome $\vec{v_0} \, // \, \vec{P}$). La $I$ è una grandezza che, in 2 dimensioni, è una grandezza scalare mentre in 3 dimensioni la situazione non è così semplice (infatti è un tensore).